In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

activity = {
        1: "Working at Computer",
        2: "Standing Up, Walking and Going updown stairs",
        3: "Standing",
        4: "Walking",
        5: "Going UpDown Stairs",
        6: "Walking and Talking with Someone",
        7: "Talking while Standin",        
    }

In [ ]:
import pandas as pd
import numpy as np
import os


path = 'C:\\Users\\Medhaja\\Desktop\\Activity Recognition from Single Chest-Mounted Accelerometer\\'
l = []
for file in [f for f in os.listdir(path) if '.csv' in f]:
    fname = file.split('_')[0]
    l.append(pd.read_csv(path+file, header=None, engine='python'))#.iloc[[0, 2], :].assign(name=fname))
    df = pd.concat(l)
df = df.drop(df.columns[0], axis=1)

In [ ]:
shape = df.shape
print('\nDataFrame Shape :', shape)
print('\nNumber of rows :', shape[0])
print('\nNumber of columns :', shape[1])

In [ ]:
df.head()

In [ ]:
df[4].value_counts().plot(kind='barh')

In [ ]:
f = df [df[4]!=0]

In [ ]:
f[4].value_counts().plot(kind='barh')

# Combine 3 and 7 as one catagory and Combine 4 and 6 as one catagory

In [ ]:
f[4].replace([7], 3, inplace=True)
f[4].replace([6],4, inplace=True)


f[4].replace([1], 0, inplace=True)
f[4].replace([2], 1, inplace=True)
f[4].replace([3], 2, inplace=True)
f[4].replace([4], 3, inplace=True)
f[4].replace([5], 4, inplace=True)

In [ ]:
f[4].value_counts().plot(kind='barh')
f.columns =['a', 'b', 'c', 'Class']

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(f, test_size=0.2)
train_df, val_df = train_test_split(f, test_size=0.2)

In [ ]:
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

In [ ]:
train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)


In [ ]:
train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)

In [ ]:
print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

In [ ]:
from keras.utils import to_categorical
from keras import utils as np_utils
one_hot_label = to_categorical(train_labels)

In [ ]:
one_hot_label

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(4, input_dim=3, activation='relu'))
model.add(keras.layers.Dense(4, activation='relu'))
model.add(keras.layers.Dense(5, activation='sigmoid'))
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(train_features, one_hot_label, epochs=2)

In [ ]:
# tl = to_categorical(test_labels)
# _ = model.evaluate(test_features,tl)

In [ ]:
preds_classes = model.predict_classes(test_features)
array = confusion_matrix(test_labels,preds_classes)

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(array, range(5), range(5))
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

In [ ]:
acc = accuracy_score(test_labels, preds_classes)
print("Accuracy is ",acc )